In [2]:
import geopandas as gpd

In [3]:
old_counties_geojson_df = gpd.read_file("counties_update.geojson")
old_counties_geojson_df.head(5)

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,merge,...,d2020-03-19,d2020-03-20,d2020-03-21,d2020-03-22,d2020-03-23,d2020-03-24,d2020-03-25,d2020-03-26,population,geometry
0,37,017,01026336,0500000US37017,37017,Bladen,06,2265887723,33010866,Bladen,...,0,0,0,0,0,0,0,0,33778,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ..."
1,37,167,01025844,0500000US37167,37167,Stanly,06,1023370459,25242751,Stanly,...,0,0,0,0,0,0,0,0,61114,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ..."
2,39,153,01074088,0500000US39153,39153,Summit,06,1069181981,18958267,Summit,...,0,0,0,0,0,0,0,0,541810,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ..."
3,42,113,01213687,0500000US42113,42113,Sullivan,06,1165338428,6617028,Sullivan,...,0,0,0,0,0,0,0,0,6177,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ..."
4,48,459,01384015,0500000US48459,48459,Upshur,06,1509910100,24878888,Upshur,...,0,0,0,0,0,0,0,0,40769,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ..."


In [4]:
old_counties_geojson_df.columns.values

array(['STATEFP', 'COUNTYFP', 'COUNTYNS', 'AFFGEOID', 'GEOID', 'NAME',
       'LSAD', 'ALAND', 'AWATER', 'merge', 'state_name', 'state_abbr',
       'confirmed_count', 'death_count', '2020-01-21', '2020-01-24',
       '2020-01-26', '2020-01-30', '2020-01-31', '2020-02-01',
       '2020-02-02', '2020-02-05', '2020-02-10', '2020-02-12',
       '2020-02-13', '2020-02-18', '2020-02-20', '2020-02-21',
       '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27',
       '2020-02-28', '2020-02-29', '2020-03-01', '2020-03-02',
       '2020-03-03', '2020-03-04', '2020-03-05', '2020-03-06',
       '2020-03-07', '2020-03-08', '2020-03-09', '2020-03-10',
       '2020-03-11', '2020-03-12', '2020-03-13', '2020-03-14',
       '2020-03-15', '2020-03-16', '2020-03-17', '2020-03-18',
       '2020-03-19', '2020-03-20', '2020-03-21', '2020-03-22',
       '2020-03-23', '2020-03-24', '2020-03-25', '2020-03-26',
       'd2020-01-21', 'd2020-01-24', 'd2020-01-26', 'd2020-01-30',
       'd2020-01-31', 'd2020-

Extract useful columns

In [5]:
counties_df = old_counties_geojson_df[['NAME','state_name','GEOID','population','geometry']]
counties_df.head(5)

,NAME,state_name,GEOID,population,geometry
0,Bladen,North Carolina,37017,33778,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ..."
1,Stanly,North Carolina,37167,61114,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ..."
2,Summit,Ohio,39153,541810,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ..."
3,Sullivan,Pennsylvania,42113,6177,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ..."
4,Upshur,Texas,48459,40769,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ..."


Check if New York City and Kansas City Exists

In [6]:
counties_df[counties_df['NAME'] == 'New York City']

,NAME,state_name,GEOID,population,geometry


In [7]:
counties_df[counties_df['NAME'] == 'Kansas City']

,NAME,state_name,GEOID,population,geometry


Add Kansas City from external GeoJSON file

In [8]:
kansas_city_geojson_df = gpd.read_file('kansas_city.geojson')
kansas_city_geojson_df.head(5)

,shape_area,nbhname,objectid,nbhid,shape_len,geometry
0,14264150.3454,Eastern 49-63,68,113,21686.6610301,"MULTIPOLYGON (((-94.56994 39.01358, -94.56996 ..."
1,7817689.84477,Broadway Gillham,28,75,11375.7151506,"MULTIPOLYGON (((-94.58030 39.06567, -94.58033 ..."
2,40471689.4662,Brown Estates,30,131,32762.6135686,"MULTIPOLYGON (((-94.51675 39.03446, -94.51583 ..."
3,34531985.9929,Calico Farms,31,160,24792.3736262,"MULTIPOLYGON (((-94.54635 38.91434, -94.54623 ..."
4,17274149.2768,CBD Downtown,32,4,17190.5486385,"MULTIPOLYGON (((-94.57232 39.10628, -94.57196 ..."


In [9]:
#Population Data get from WIKI: https://en.wikipedia.org/wiki/Kansas_City,_Missouri
#According to the U.S. Census Bureau, the city had an estimated population of 491,918 in 2018
kansas_geometry = kansas_city_geojson_df.geometry.unary_union
counties_df = counties_df.append(
    {
        'NAME':'Kansas City',
        'state_name':'Missouri',
        'GEOID':'00000',
        'population':491918,
        'geometry':kansas_geometry
    },ignore_index=True)

In [10]:
counties_df[counties_df['NAME'] == 'Kansas City']

,NAME,state_name,GEOID,population,geometry
3220,Kansas City,Missouri,00000,491918,"POLYGON ((-94.44849 38.93168, -94.44869 38.928..."


Add New York City by merging 5 counties

In [11]:
new_york_city_df = counties_df[((counties_df['NAME'] == 'New York') 
            | (counties_df['NAME'] == 'Kings') 
            | (counties_df['NAME'] == 'Queens') 
            | (counties_df['NAME'] == 'Bronx')
            | (counties_df['NAME'] == 'Richmond'))
            & (counties_df['state_name'] == 'New York')]


In [12]:
nyc_geometry = new_york_city_df.geometry.unary_union

In [13]:
counties_df = counties_df.append(
    {
        'NAME':'New York City',
        'state_name':'New York',
        'GEOID':'00000',
        'population':new_york_city_df['population'].sum(),
        'geometry':nyc_geometry,
    },ignore_index=True)

In [14]:
counties_df[counties_df['NAME'] == 'New York City']

,NAME,state_name,GEOID,population,geometry
3221,New York City,New York,00000,8443713,"POLYGON ((-74.24921 40.54506, -74.21684 40.558..."


In [17]:
counties_df[((counties_df['NAME'] == 'New York') 
            | (counties_df['NAME'] == 'Kings') 
            | (counties_df['NAME'] == 'Queens') 
            | (counties_df['NAME'] == 'Bronx')
            | (counties_df['NAME'] == 'Richmond'))
            & (counties_df['state_name'] == 'New York')]

,NAME,state_name,GEOID,population,geometry
39,Bronx,New York,36005,1437872,"MULTIPOLYGON (((-73.94748 40.85777, -73.93808 ..."
1139,Queens,New York,36081,2298513,"MULTIPOLYGON (((-73.96248 40.73680, -73.92100 ..."
1492,New York,New York,36061,1632480,"MULTIPOLYGON (((-74.02564 40.73322, -74.02349 ..."
1953,Kings,New York,36047,2600747,"MULTIPOLYGON (((-74.06772 40.67038, -74.04731 ..."
2031,Richmond,New York,36085,474101,"MULTIPOLYGON (((-74.24921 40.54506, -74.21684 ..."


In [21]:
counties_df = counties_df.drop(index = [39,1139,1492,1953,2031])

In [22]:
counties_df[((counties_df['NAME'] == 'New York') 
            | (counties_df['NAME'] == 'Kings') 
            | (counties_df['NAME'] == 'Queens') 
            | (counties_df['NAME'] == 'Bronx')
            | (counties_df['NAME'] == 'Richmond'))
            & (counties_df['state_name'] == 'New York')]

,NAME,state_name,GEOID,population,geometry


In [23]:
counties_df.to_file("counties_update_new.geojson", driver='GeoJSON', encoding='utf-8')
print("done")


done
